In [2]:
val weatherRdd = sc.textFile("wasb://bom-weatherobs@sa1govhackdatablobs.blob.core.windows.net/")

weatherRdd: org.apache.spark.rdd.RDD[String] = wasb://bom-weatherobs@sa1govhackdatablobs.blob.core.windows.net/ MapPartitionsRDD[1] at textFile at <console>:15

In [3]:
weatherRdd.count

res1: Long = 16476942

In [4]:
weatherRdd.take(1)

res2: Array[String] = Array(station_number,area_code,parameter,valid_start,valid_end,value,unit,statistic,instantaneous,level,qc_valid_minutes,qc_valid_minutes_start,qc_valid_minutes_end)

In [5]:
weatherRdd.filter(_ != "station_number,area_code,parameter,valid_start,valid_end,value,unit,statistic,instantaneous,level,qc_valid_minutes,qc_valid_minutes_start,qc_valid_minutes_end").take(1)

res3: Array[String] = Array(1006,WA_PT174,AIR_TEMP,1430409600,1430409600,19.8,Celsius,first,True,SFC,,1430409600,)

In [10]:
import java.sql.Timestamp

case class weatherReading(stationNumber: Int, areaCode: String, parameter: String, validStart: Timestamp, validEnd: Timestamp, value: Double, unit: String, statistic: String, instantaneous: Boolean, level: String)

defined class weatherReading

In [11]:
import sqlContext.implicits._

val weatherDf = weatherRdd.filter(_ != "station_number,area_code,parameter,valid_start,valid_end,value,unit,statistic,instantaneous,level,qc_valid_minutes,qc_valid_minutes_start,qc_valid_minutes_end").map(line => {
    
    val cols = line.split(",")
    
    weatherReading(cols(0).toInt, cols(1), cols(2), new Timestamp(cols(3).toLong * 1000), new Timestamp(cols(4).toLong * 1000), cols(5).toDouble, cols(6), cols(7), cols(8).toBoolean, cols(9))
    
}).toDF

weatherDf: org.apache.spark.sql.DataFrame = [stationNumber: int, areaCode: string, parameter: string, validStart: timestamp, validEnd: timestamp, value: double, unit: string, statistic: string, instantaneous: boolean, level: string]

In [15]:
import org.apache.spark.sql.SaveMode

import org.apache.spark.sql.SaveMode

In [16]:
weatherDf.write.mode(SaveMode.Overwrite).parquet("wasb://weatherobs@sa1govhackdatablobs.blob.core.windows.net/parquet/")

In [17]:
%%sql -q
CREATE EXTERNAL TABLE IF NOT EXISTS weatherobs ( 
    stationNumber Int, 
    areaCode String,
    parameter String,
    validStart Timestamp,
    validEnd Timestamp,
    value Double,
    unit String,
    statistic String,
    instantaneous Boolean,
    level String) 
    STORED AS PARQUET
    LOCATION 'wasb://weatherobs@sa1govhackdatablobs.blob.core.windows.net/parquet/'